# Installing Dependencies

In [8]:
import sys

# Installing numpy
!{sys.executable} -m pip install numpy

# Installing pandas
!{sys.executable} -m pip install pandas

# Installing matplotlib
!{sys.executable} -m pip install matplotlib

# Installing statsmodels
!{sys.executable} -m pip install statsmodels

#Installing tqdm
!{sys.executable} -m pip install tqdm


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Imports

In [48]:
# Import Statistic Libraries
import numpy                    as np
import pandas                   as pd
import matplotlib.pyplot        as plt

# Import Utility Libraries
from typing                     import List, Dict, TypedDict
from tqdm                       import tqdm
from matplotlib.lines           import Line2D

# Algorythm

In [49]:
users_df: pd.DataFrame = pd.read_csv('../data/users.csv')
requests_df: pd.DataFrame  = pd.read_csv('../data/requests.csv')
listings_df: pd.DataFrame  = pd.read_csv('../data/listings.csv')

class UserRequest(TypedDict):
    City: str
    MinPrice: float
    MaxPrice: float
    MinSpace: float
    MaxSpace: float

In [50]:
def match_listings(user_request: pd.DataFrame, listings_df: pd.DataFrame):
    matching_df = listings_df[
        (listings_df['City'] == user_request['City']) &
        (listings_df['Price'] >= user_request['MinPrice'] if not pd.isna(user_request['MinPrice']) else True) &
        (listings_df['Price'] <= user_request['MaxPrice'] if not pd.isna(user_request['MaxPrice']) else True) &
        (listings_df['Area'] >= user_request['MinSpace'] if not pd.isna(user_request['MinSpace']) else True) &
        (listings_df['Area'] <= user_request['MaxSpace']if not pd.isna(user_request['MaxSpace']) else True)
    ]

    return matching_df

In [53]:
user_request: UserRequest = requests_df.iloc[1].to_dict()
print(user_request)
matching_results = match_listings(user_request, listings_df)
print(matching_results)


{'City': 'Groningen', 'MinPrice': 400.0, 'MaxPrice': 1200.0, 'MinSpace': 18, 'MaxSpace': nan, 'email': 'daniel@daniel.com'}
Empty DataFrame
Columns: [City, Street, Price, Area, email]
Index: []
